In [37]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [38]:
df = pd.read_csv('../data/process_v1/process_data_6.csv')
df = df[df['city'] == 0]

In [39]:
df['is_street_house'] = (df['accessibility'] == 2).astype(np.int32)

In [40]:
df['landSize_ratio'] = df['landSize'] / df['acreage']

In [41]:
mean_land_size_df = df.groupby('administrative_genre')['landSize'].mean().reset_index().rename(columns = {'landSize': 'meanLandSize'})

df = df.merge(mean_land_size_df, how='left', on = 'administrative_genre')

df['landSize_ratio_with_administrative_genre'] = df['landSize'] / df['meanLandSize']
df['acreage_ratio_with_meanLandSize'] = df['acreage'] / df['meanLandSize']


del df['meanLandSize']

def partition_for_alley_width(alley_width):
    if math.isnan(alley_width):
        return np.nan
    if alley_width <= 2.5:
        return 1
    if alley_width <= 5:
        return 2
    if alley_width <= 7:
        return 3
    if alley_width <= 8:
        return 4

In [42]:
{'fitThreeCars': 0,
 'fitOneCarAndOneMotorbike': 1,
 'notInTheAlley': 2,
 'parkCar': 3,
 'theBottleNeckPoint': 4,
 'fitTwoCars': 5,
 'narrorRoad': 6}

{'fitThreeCars': 0,
 'fitOneCarAndOneMotorbike': 1,
 'notInTheAlley': 2,
 'parkCar': 3,
 'theBottleNeckPoint': 4,
 'fitTwoCars': 5,
 'narrorRoad': 6}

In [43]:
cat_cols = ['is_street_house', 'ward', 'street', 'district', 'typeOfRealEstate', 'houseDirection', 'accessibility', 'certificateOfLandUseRight'] + [c for c in df.columns.tolist() if '_district' in c or '_ward' in c or '_street' in c]
target_cols = ['target']
remove_cols = ['price', 'description', 'city'] + [c for c in df.columns if 'distance_hcm' in c]
time_cols = ['time']
num_cols = [c for c in df.columns.tolist() if c not in (cat_cols + target_cols + remove_cols + time_cols)]

cat_cols = list(set(cat_cols))
target_cols = list(set(target_cols))
remove_cols = list(set(remove_cols))
num_cols = list(set(num_cols))

print(len(cat_cols), len(target_cols), len(remove_cols), len(num_cols))

37 1 52 147


In [44]:
df.columns.tolist()

['numberOfFloors',
 'numberOfBathRooms',
 'numberOfBedRooms',
 'certificateOfLandUseRight',
 'ward',
 'street',
 'district',
 'city',
 'lat',
 'lon',
 'typeOfRealEstate',
 'frontWidth',
 'endWidth',
 'facade',
 'houseDirection',
 'landSize',
 'price',
 'accessibility',
 'description',
 'time',
 'w',
 'h',
 'num_of_university_in_500m_radius',
 'num_of_fuel_in_500m_radius',
 'num_of_cafe_in_500m_radius',
 'num_of_parking_in_500m_radius',
 'num_of_parking_entrance_in_500m_radius',
 'num_of_fast_food_in_500m_radius',
 'num_of_marketplace_in_500m_radius',
 'num_of_restaurant_in_500m_radius',
 'num_of_hospital_in_500m_radius',
 'num_of_school_in_500m_radius',
 'num_of_kindergarten_in_500m_radius',
 'num_of_townhall - community_centre_in_500m_radius',
 'num_of_police_in_500m_radius',
 'num_of_place_of_worship_in_500m_radius',
 'num_of_bank_in_500m_radius',
 'num_of_atm_in_500m_radius',
 'num_of_university_in_1000m_radius',
 'num_of_fuel_in_1000m_radius',
 'num_of_cafe_in_1000m_radius',
 'num_

In [45]:
cat_cols

['nearest_2_district',
 'nearest_3_street',
 'nearest_6_street',
 'nearest_1_district',
 'nearest_8_district',
 'accessibility',
 'nearest_7_district',
 'typeOfRealEstate',
 'nearest_1_street',
 'nearest_6_ward',
 'nearest_0_district',
 'nearest_2_ward',
 'nearest_0_street',
 'nearest_4_district',
 'nearest_7_street',
 'nearest_2_street',
 'nearest_8_ward',
 'district',
 'houseDirection',
 'nearest_1_ward',
 'distance_district',
 'ward',
 'nearest_5_district',
 'nearest_5_street',
 'nearest_5_ward',
 'nearest_6_district',
 'is_street_house',
 'nearest_4_street',
 'nearest_4_ward',
 'nearest_8_street',
 'nearest_0_ward',
 'nearest_3_ward',
 'nearest_3_district',
 'street',
 'certificateOfLandUseRight',
 'nearest_7_ward',
 'num_of_ward']

In [46]:
num_cols

['endWidth',
 'distance_nearest_2',
 'num_of_hospital_in_1000m_radius',
 'num_of_kindergarten_in_500m_radius',
 'distance_hn_Nguyễn Thị Minh Khai',
 'num_of_fuel_in_1000m_radius',
 'distance_hn_Đường Lê Anh Xuân, quận 1',
 'distance_nearest_0',
 'num_of_townhall - community_centre_in_500m_radius',
 'facade',
 'num_of_fast_food_in_2000m_radius',
 'num_of_cafe_in_2000m_radius',
 'num_of_marketplace_in_500m_radius',
 'acreage',
 'distance_hn_Công viên 30-4',
 'distance_nearest_1',
 'num_of_university_in_2000m_radius',
 'numberOfFloors',
 'distance_hn_Nguyễn Tri Phương',
 'distance_hn_Đào Trí',
 'numberOfBathRooms',
 'distance_hn_Công viên khu đô thị Sala',
 'distance_hn_Công viên Gia Định',
 'nearest_4_lon',
 'landSize_ratio_with_administrative_genre',
 'frontWidth',
 'num_of_atm_in_500m_radius',
 'lat',
 'num_of_place_of_worship_in_500m_radius',
 'nearest_1_lat',
 'distance_nearest_8',
 'district_lat',
 'distance_hn_TTTM Saigon Centre (Takashimaya)',
 'num_of_hospital_in_2000m_radius',
 

In [47]:
print(len(num_cols))

147


In [48]:
df[cat_cols] = df[cat_cols].fillna(100)
df[cat_cols] = df[cat_cols].astype(np.int32)

In [49]:
df = df[cat_cols + num_cols + time_cols + target_cols]

In [50]:
df = df[df[target_cols[0]] != np.inf]
df = df[df[target_cols[0]] != -np.inf]

In [51]:
df.shape

(49052, 186)

In [52]:
df = df[df[target_cols[0]] >= 10]
df = df[df[target_cols[0]] < 100 * 10]
# df = pd.concat([df[df['w'] < 50], df[df['w'].isnull()]])
# df = pd.concat([df[df['h'] < 50], df[df['w'].isnull()]])

In [53]:
df.to_csv('../data/process_v1/process_data_7_hcm.csv', index = False)

In [54]:
num_cols

['endWidth',
 'distance_nearest_2',
 'num_of_hospital_in_1000m_radius',
 'num_of_kindergarten_in_500m_radius',
 'distance_hn_Nguyễn Thị Minh Khai',
 'num_of_fuel_in_1000m_radius',
 'distance_hn_Đường Lê Anh Xuân, quận 1',
 'distance_nearest_0',
 'num_of_townhall - community_centre_in_500m_radius',
 'facade',
 'num_of_fast_food_in_2000m_radius',
 'num_of_cafe_in_2000m_radius',
 'num_of_marketplace_in_500m_radius',
 'acreage',
 'distance_hn_Công viên 30-4',
 'distance_nearest_1',
 'num_of_university_in_2000m_radius',
 'numberOfFloors',
 'distance_hn_Nguyễn Tri Phương',
 'distance_hn_Đào Trí',
 'numberOfBathRooms',
 'distance_hn_Công viên khu đô thị Sala',
 'distance_hn_Công viên Gia Định',
 'nearest_4_lon',
 'landSize_ratio_with_administrative_genre',
 'frontWidth',
 'num_of_atm_in_500m_radius',
 'lat',
 'num_of_place_of_worship_in_500m_radius',
 'nearest_1_lat',
 'distance_nearest_8',
 'district_lat',
 'distance_hn_TTTM Saigon Centre (Takashimaya)',
 'num_of_hospital_in_2000m_radius',
 

In [55]:
cat_cols

['nearest_2_district',
 'nearest_3_street',
 'nearest_6_street',
 'nearest_1_district',
 'nearest_8_district',
 'accessibility',
 'nearest_7_district',
 'typeOfRealEstate',
 'nearest_1_street',
 'nearest_6_ward',
 'nearest_0_district',
 'nearest_2_ward',
 'nearest_0_street',
 'nearest_4_district',
 'nearest_7_street',
 'nearest_2_street',
 'nearest_8_ward',
 'district',
 'houseDirection',
 'nearest_1_ward',
 'distance_district',
 'ward',
 'nearest_5_district',
 'nearest_5_street',
 'nearest_5_ward',
 'nearest_6_district',
 'is_street_house',
 'nearest_4_street',
 'nearest_4_ward',
 'nearest_8_street',
 'nearest_0_ward',
 'nearest_3_ward',
 'nearest_3_district',
 'street',
 'certificateOfLandUseRight',
 'nearest_7_ward',
 'num_of_ward']